# Titanic dataset classification, from Kaggle

Seemed like an interesting way to learn a little about classification. One thing that's nice about a competition is that you can actually look at how well your algorithm performs relative to others. How else do you findout if you're making a strong model? 


In [ ]:
## imports the essentials

import pandas as pd
import numpy as np

## open the dataset and explore

df = pd.read_csv('train.csv')
df.head()

In [ ]:
print(df.shape)

## We need to impute some stuff.
print(df.isna().sum())

## Cabin
df['Cabin'].unique()

## This is weird... the passenger fares are to very high precision.
#print(df['Fare'].unique())
df['Age'].describe()

In [ ]:
## This is weird... the passenger fares are to very high precision.
## For families, they are sums.
#print(df['Fare'].unique())
df['Age'].describe()
df[['Pclass','Fare']][df['Pclass'] == 1] ##Though correlated, these are not the same thing!
pd.Series(df['Fare'].unique()).describe()
df.sort_values(by='Ticket').head(10)

In [ ]:
df['Ticket'].value_counts().loc['347082']

In [150]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler



def feature_engineer(df):
    '''
    All feature engineering to the dataframe is done here and returned, as a copy. 
    '''
    working_data = df.copy()
    ages = SimpleImputer(strategy='mean', add_indicator=True).fit_transform(df[['Age',]])
    working_data['Age'] = ages[:,0]
    working_data['Age Unknown'] = ages[:,1]
    working_data['Sex'] = df['Sex'].apply(lambda x: x == 'female')
    working_data['Cabin'] = df['Cabin'].isna()
    working_data['Family Size'] = df['SibSp'] + df['Parch']

    ## The things below didn't improve the performance of my ensembles. 
    #working_data['Embarked'] = df['Embarked'].map({'S':0,'C':1,'Q':2}).astype('category')
    #df['cabin group'] = df['Cabin'].apply(lambda x: x[0])
    #working_data['Age Group'] = pd.cut(working_data['Age'], bins = [0,15,30,45,60,75,120], labels=False)
    #working_data['is child'] = working_data['Age'].apply(lambda x: x < 18)

    ## Data exploration suggests that many passengers buy one ticket for multiple persons.
    ## We divid each fare by the number of people who have a matching ticket number and fill NaN with the mean.
    working_data['Fare'] = working_data.apply(lambda x: x['Fare']/working_data['Ticket'].value_counts().loc[x['Ticket']], axis = 1)
    working_data['Fare'].fillna(working_data['Fare'].mean(),inplace=True)

    ## Cut the fare data into quartitles
    working_data['Fare Group'] = pd.qcut(working_data['Fare'], 4, labels=False, )
    
    ## Select the data you want to use. Based on some experimentation below.
    working_data = working_data.drop(['Name','Ticket','Embarked','Family Size','Cabin','Age Unknown','Fare'],axis=1)
    return working_data
working_data = feature_engineer(df)


In [151]:
## by playing with the matching below, we can look to see if any variable stands out 
## as relevant to different groups.
temp = working_data
#temp = temp[(temp['Sex']==0)]
#temp = temp[temp['SibSp'] == 0]
print(temp.shape)
temp.corr()

(891, 8)


,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare Group
PassengerId,1.000000,-0.005007,-0.035144,-0.042939,0.033207,-0.057527,-0.001652,-0.009839
Survived,-0.005007,1.000000,-0.338481,0.543351,-0.069809,-0.035322,0.081629,0.305756
Pclass,-0.035144,-0.338481,1.000000,-0.131900,-0.331339,0.083081,0.018443,-0.812160
Sex,-0.042939,0.543351,-0.131900,1.000000,-0.084153,0.114631,0.245489,0.134794
Age,0.033207,-0.069809,-0.331339,-0.084153,1.000000,-0.232625,-0.179191,0.304921
SibSp,-0.057527,-0.035322,0.083081,0.114631,-0.232625,1.000000,0.414838,-0.013418
Parch,-0.001652,0.081629,0.018443,0.245489,-0.179191,0.414838,1.000000,-0.032146
Fare Group,-0.009839,0.305756,-0.812160,0.134794,0.304921,-0.013418,-0.032146,1.000000


In [152]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

X_train, X_test, y_train, y_test = train_test_split(working_data.drop(['PassengerId','Survived'], axis=1),
                    working_data['Survived'],
                    train_size = .666)

X_train.shape
X_test.shape

(298, 6)

## Let's establish a baseline

A decision tree is quick and lets you know if what you've got.

In [164]:
tree_clf = DecisionTreeClassifier(max_depth = 1, min_samples_leaf=3)
simple = ['Sex']## use the feature that correlate most obviously with survival.
tree_clf.fit(X_train[simple],y_train)
print(tree_clf.score(X_train[simple],y_train))
print(tree_clf.score(X_test[simple],y_test))

0.7824620573355818
0.7953020134228188


In [121]:
forest_clf = RandomForestClassifier(n_estimators=30, max_depth = 6)
#forest_clf = RandomForestClassifier()

In [166]:
def clf_eval_summary(clf,X=X_test,y=y_test, X_=X_train, y_=y_train):
    print("Training score: {}".format(clf.score(X_,y_)))
    print("Test score: {}".format(clf.score(X,y)))
    print("Confusion:")
    print(confusion_matrix(y,clf.predict(X)))
    

In [171]:
## The following check whether adding a feature improves model performance.


from sklearn.metrics import confusion_matrix

diffs = []
times = 5
for i in range(times):
    simple = ['Sex','Fare Group','Age']
    forest_clf.fit(X_train[simple], y_train)
    simpler = forest_clf.score(X_test[simple], y_test)
    forest_clf.fit(X_train,y_train)
    diffs.append(forest_clf.score(X_test, y_test) - simpler)
    #print(diffs[-1], simpler)
    #
forest_clf.fit(X_train[simple], y_train)
clf_eval_summary(forest_clf, X=X_test[simple], X_=X_train[simple])
forest_clf.fit(X_train, y_train)
clf_eval_summary(forest_clf)    
print("Model improvement over a simple model: {:.4f}".format(sum(diffs)/times))

Training score: 0.8482293423271501
Test score: 0.8154362416107382
Confusion:
[[172  25]
 [ 30  71]]
Training score: 0.8549747048903878
Test score: 0.8221476510067114
Confusion:
[[179  18]
 [ 35  66]]
Model improvement over a simple model: 0.0201


## Some validation and tuning of the Random Forest

In [169]:
##Cross validate the model to see how it looks.
from sklearn.model_selection import cross_val_score

cross_val_score(forest_clf,X_train,y_train).mean()


0.8110810425865262

In [ ]:
from sklearn.model_selection import GridSearchCV

parameters = {'n_estimators':[50,100],'max_depth' : [5,7,9]}
forest_search = GridSearchCV(forest_clf, param_grid=parameters, return_train_score=True)

In [ ]:
forest_search.fit(X_train,y_train)

In [ ]:
forest_search.cv_results_

In [ ]:
## It's worth noting that these estimators are mostly within a standard deviation or so
## from each other in test performance. Without a bigger data set, it's hard to validate these parameters 
## against each other.

best = forest_search.best_estimator_
best

## Other ensembles?

I tried other models. Sometimes they outperformed the Random Forest, but they never improved the kaggle test set performance.

In [78]:
from sklearn.ensemble import GradientBoostingClassifier

gradient_clf = GradientBoostingClassifier(n_estimators=100, max_depth=2)
gradient_clf.fit(X_train,y_train)

GradientBoostingClassifier(max_depth=2)

In [79]:
clf_eval_summary(gradient_clf)
cross_val_score(gradient_clf,X_train,y_train).mean()

Training score: 0.8696629213483146
Test score: 0.8026905829596412
Confusion:
[[244  34]
 [ 54 114]]


0.8089887640449438

In [ ]:
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline

svc = Pipeline(steps=[('scaler',StandardScaler()),('svc',SVC(C=1))])
svc.fit(X_train,y_train)
cross_val_score(svc, X_train, y_train).mean()

In [ ]:
parameters = {'svc__C':[1,10,100],'svc__kernel':['rbf','poly']}

svc_search = GridSearchCV(svc, param_grid=parameters, return_train_score=True)

In [ ]:
svc_search.fit(X_train,y_train)
svc_search.cv_results_

## This was a moment

I will let the cat out of the bag and say that a NN was the strongest model I made with only a little tuning. As you can see from the comments below, I was skeptical that a NN could outperform the other models I tried with a relatively small data set. In fact, I was so skeptical that I spent quite awhile feature engineering and tuning other models before I tried these. 

In [158]:
## I'm getting annoyed that my best classifier seems to top out around .785 on Kaggle and that improvements
## on my dev set aren't translating to the test set. 

## I really don't think there's enough data to support a neural net.
from sklearn.neural_network import MLPClassifier

## The parameters selected here are the results of the tuning that I did a few cells down.
mlp_clf = Pipeline(steps=[('scaler',StandardScaler()),('NN', MLPClassifier(alpha=0.01, hidden_layer_sizes=15,
                               learning_rate_init=0.005))])

mlp_clf.fit(X_train,y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('NN',
                 MLPClassifier(alpha=0.01, hidden_layer_sizes=15,
                               learning_rate_init=0.005))])

In [155]:
parameters = {'NN__learning_rate_init':[.005,.01], 'NN__hidden_layer_sizes': [10, 15], 'NN__alpha':[.003,.01,.03]}
mlp_grid = GridSearchCV(mlp_clf, param_grid=parameters, return_train_score=True)
mlp_grid.fit(X_train,y_train)

GridSearchCV(estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('NN',
                                        MLPClassifier(hidden_layer_sizes=10,
                                                      learning_rate_init=0.03))]),
             param_grid={'NN__alpha': [0.01, 0.03],
                         'NN__hidden_layer_sizes': [15],
                         'NN__learning_rate_init': [0.005, 0.01]},
             return_train_score=True)

In [157]:
mlp_grid.cv_results_
mlp_grid.best_estimator_

Pipeline(steps=[('scaler', StandardScaler()),
                ('NN',
                 MLPClassifier(alpha=0.01, hidden_layer_sizes=15,
                               learning_rate_init=0.005))])

In [159]:
df1 = df.copy()
results = feature_engineer(pd.read_csv('test.csv'))
print(results.isna().sum()) ## double check NaN values in the output.

mlp_clf.fit(X_train,y_train) ## makes sure the model is properly fitted. 
results['Survived'] = mlp_clf.predict(results.drop('PassengerId', axis=1))
old_results = pd.read_csv('final_results.csv')
results[['PassengerId','Survived']].set_index('PassengerId').to_csv('final_results.csv')
results

PassengerId    0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare Group     0
dtype: int64


,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare Group,Survived
0,892,3,False,34.50000,0,0,0,0
1,893,3,True,47.00000,1,0,0,0
2,894,2,False,62.00000,0,0,1,0
3,895,3,False,27.00000,0,0,1,0
4,896,3,True,22.00000,1,1,1,1
...,...,...,...,...,...,...,...,...
413,1305,3,False,30.27259,0,0,1,0
414,1306,1,True,39.00000,0,0,3,1
415,1307,3,False,38.50000,0,0,0,0
416,1308,3,False,30.27259,0,0,1,0


In [160]:
np.abs((results['Survived'] - old_results['Survived'])).sum() ## check to see if the new results are much different from the old.

21

## The MLP scored .79904

As a number of people have mentioned in the discussion of the data, it's unfortunate that the leaderboard hasn't been reset. The actual Titanic data is publicly available. This means that there are lots of people with perfect scores. _But the data isn't really that powerful._ 

Anyway, we see below that this score places my in the 96th percentile of scores, once we get rid of obviously bad models and too good to be true ones.

In [188]:
leaders = pd.read_csv('titanic-publicleaderboard.csv')
leaders = leaders[(leaders['Score'] < .95) & (leaders['Score'] > .7)].sort_values(by='Score')
leaders['Rank'] = leaders['Score'].rank(method='first', ascending=False)
leaders['Percentile'] = (leaders.shape[0]-leaders['Rank'])/leaders.shape[0]

In [189]:
leaders.head()

,TeamId,TeamName,SubmissionDate,Score,Rank,Percentile
10055,5610225,Mika Klinger,2020-10-04 20:03:42,0.70095,16115.0,0.000434
10368,5615512,samran lushan,2020-10-15 15:13:58,0.70095,16116.0,0.000372
8817,5579954,Puranjan Sojitra,2020-09-29 11:18:43,0.70095,16117.0,0.000310
8639,5576800,shizuki.okumura,2020-09-26 11:09:45,0.70095,16118.0,0.000248
501,1700801,Khalid Jeffal,2020-11-05 16:37:41,0.70095,16119.0,0.000186


In [190]:
leaders[leaders['TeamName'] == 'sjlenhart']

,TeamId,TeamName,SubmissionDate,Score,Rank,Percentile
17222,5796517,sjlenhart,2020-11-10 21:30:40,0.79904,526.0,0.967374
